In [3]:
import sys
import os
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import json

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [4]:
# average volume, top 10 levels
processed_dir = os.path.join(project_root, 'data', 'processed')
tickers = [name for name in os.listdir(processed_dir) if os.path.isdir(os.path.join(processed_dir, name))]
print(tickers)
time_col = 0

avg_diffs = {}
for ticker in tickers:
    ob_dir = os.path.join(processed_dir, ticker, 'orderbooks', 'scaled')
    if os.path.isdir(ob_dir):
        csv_files = [f for f in os.listdir(ob_dir) if f.endswith('.csv')]
        if csv_files:
            first_csv = sorted(csv_files)[0]
            csv_path = os.path.join(ob_dir, first_csv)
            df_ob = pl.read_csv(csv_path).to_pandas()
            # Compute the average difference between consecutive values in column 1
            col1 = df_ob.iloc[:, time_col]
            avg_diff = col1.diff().abs().mean()
            print(f"Ticker: {ticker}, Average Update difference: {avg_diff}")
            avg_diffs[ticker] = avg_diff
        else:
            print(f"No CSV files found in {ob_dir}")
    else:
        print(f"Directory does not exist: {ob_dir}")
    

['AAPL', 'CSCO', 'NFLX', 'NVDA', 'TSLA']
Ticker: AAPL, Average Update difference: 0.016324748125661145
Ticker: CSCO, Average Update difference: 0.04036218581027897
Ticker: NFLX, Average Update difference: 0.28448296189298344
Ticker: NVDA, Average Update difference: 0.008002265498323332
Ticker: TSLA, Average Update difference: 0.09110526379624949
